In [4]:
from mshr import *
import numpy as np
from dolfin import *
import matplotlib.pyplot as plt

# Form compiler options
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["optimize"] = True

In [5]:
mesh = RectangleMesh(Point(-1,-1), Point(1,1), 64, 64)

def boundary1(x, on_boundary):
    cond1 = on_boundary and near(x[0], -1)
    cond2 = x[1] >= -1.0/3.0 and x[1] <= 1.0/3.0
    return cond1 and cond2

def boundary2(x, on_boundary):
    cond1 = on_boundary
    cond2 = not (x[1] >= -1.0/3.0 and x[1] <= 1.0/3.0 and near(x[0], -1))
    return cond1 and cond2

In [ ]:
alpha_m = Constant(0.2)
alpha_f = Constant(0.4)
gamma   = Constant(0.5+alpha_f-alpha_m)
beta    = Constant((gamma+0.5)**2/4.)

# Time-stepping parameters
T       = 4.0
Nsteps  = 50
dt = Constant(T/Nsteps)

In [6]:
"""
Define the weighting function and trial solution spaces
"""
order = 1 # order of basis functions

V = FunctionSpace(mesh, "CG", order)

# Test and trial functions
du = TrialFunction(V)
u_ = TestFunction(V)

# current, unknown displacement
u = Function(V, name="Displacement")
# previous time step functions
u_old = Function(V)
v_old = Function(V)
a_old = Function(V)

In [7]:

# set the source to a constant for now
u_D1 = Expression("sin(4*pi*t)", degree = order, t = 0.0)
u_D2 = Constant(0.0)

bc1 = DirichletBC(V, u_D1, boundary1)
bc2 = DirichletBC(V, u_D2, boundary2)
bc = [bc1, bc2]

In [ ]:
# Update formula for acceleration
# a = 1/(2*beta)*((u - u0 - v0*dt)/(0.5*dt*dt) - (1-2*beta)*a0)
def update_a(u, u_old, v_old, a_old, ufl=True):
    if ufl:
        dt_ = dt
        beta_ = beta
    else:
        dt_ = float(dt)
        beta_ = float(beta)
    return (u-u_old-dt_*v_old)/beta_/dt_**2 - (1-2*beta_)/2/beta_*a_old

# Update formula for velocity
# v = dt * ((1-gamma)*a0 + gamma*a) + v0
def update_v(a, u_old, v_old, a_old, ufl=True):
    if ufl:
        dt_ = dt
        gamma_ = gamma
    else:
        dt_ = float(dt)
        gamma_ = float(gamma)
    return v_old + dt_*((1-gamma_)*a_old + gamma_*a)

def update_fields(u, u_old, v_old, a_old):
    """Update fields at the end of each time step."""

    # Get vectors (references)
    u_vec, u0_vec  = u.vector(), u_old.vector()
    v0_vec, a0_vec = v_old.vector(), a_old.vector()

    # use update functions using vector arguments
    a_vec = update_a(u_vec, u0_vec, v0_vec, a0_vec, ufl=False)
    v_vec = update_v(a_vec, u0_vec, v0_vec, a0_vec, ufl=False)

    # Update (u_old <- u)
    v_old.vector()[:], a_old.vector()[:] = v_vec, a_vec
    u_old.vector()[:] = u.vector()

In [1]:
# stiffness matrix
def k(function):
    return inner(grad(function), grad(du))*dx


In [ ]:
a_new = update_a(du, u_old, v_old, a_old, ufl=True)
v_new = update_v(a_new, u_old, v_old, a_old, ufl=True)
res = m(avg(a_old, a_new, alpha_m), u_) + k(u_old), du, alpha_f), u_)
a_form = lhs(res)
L_form = rhs(res)